## Chapter 7: Functional Programming

This chapter covers the introduction to functional programming.  In short, functional programming languages have functions as a important part of the language and apply and compose functions to create programs.

### 7.1: Functional vs. Non-functional forms

Here is a simple example about create an array from another array in both non-functional and functional forms:

In [ ]:
v=collect(1:5)

This is the traditional (non-functional form) of making a new array of squares of the original.

In [ ]:
v2=zeros(Int,5) # this an array of zeros of length 5 
for i=1:5
  v2[i]=v[i]^2 
end
v2

Here are some functional form versions instead:

In [ ]:
f(x)=x^2
map(f,v)

This is the same with an anonymous function

In [ ]:
map(x->x^2,v)

And recall that we did this in the previous chapter with:

In [ ]:
v.^2

The `map` function starts with an array, applies a function to each element and returns the new array.  Array an input and array as output.


#### Exercise
create a vector from 1 to 10 and 1) make a new array of `[1, 0.5, 0.333, 0.25, ...]`  and 2) make another with `[1 1//2 1//3 1//4 ...`] using the `map` function and the anonymous function notation.

### 7.2: Reducing an array

Another very common idea with arrays is to start with an array and reduce it to a single number.  The classic example is a sum. 

In [ ]:
A=[1,2,3,4,5]

To do this, we use the `reduce` function, which has the same form as the `map` function except the function has two arguments. 

In [ ]:
reduce((x,y)-> x+y, A)

To see really what's going on, here's a debugging version of that:

In [ ]:
function f(tot,val)
  @show tot,val
  tot+val
end

In [ ]:
reduce(f,A)

Notice that the first variable keeps the running sum.  Also, notice that the first call, when `val=1` is missing.  This is because the first value needs the function applied to something (it's binary).  If you don't put in an initial value, (often) it is initialized to 0 and a sum is applied on the first step.  Here's more what's going on.

In [ ]:
reduce(f,A,init=0)

In [ ]:
reduce((x,y)->x*y,A)

If we have an array that we want to count the number of position element in the array, then here's a nice way to do this with `reduce`.  Again, this array as an input, value as an output.

In [ ]:
numPos(arr::Array{Int64,1}) = reduce((num,val) -> val > 0 ? num+1 : num, arr, init=0)

In [ ]:
numPos([-3,5,8,-2,11])

#### Exercise

Use reduce to take an array of strings, say `["The","dog","bit","the","cat"]` and concatenate all of the strings. (Can you figure out how to add a space between words?)

#### Summary of reduce

If you have an 1D array (vector) of numbers/strings (or anything) and you want to summarize the vector with a single value (number, string), you can often use reduced to do this. 

### 7.3: The mapreduce function

A super-handy funtion is the mapreduce function.  It's a combination of mapping an array to another array and then reducing all in one.  Consider the following that takes an array, squares each element and then sums:

In [ ]:
mapreduce(x->x^2,+,[1,2,3])

Since the function is the sum, this is also built into the sum command:

In [ ]:
sum(x->x^2,[1,2,3])

This example, takes an array of strings, and find the average word length:

In [ ]:
mapreduce(str->length(str),+,["This","is","a","very","boring","sentence"])/6

### 7.4: Mapping a Function over an 2D array

If we have a 2D array, we can map over the rows or columns of the array using the `mapslices` function.  The result is a 1D array (vector).

In [ ]:
A=[i+j for i=1:10,j=1:3]

This is the column sums

In [ ]:
mapslices(sum,A,dims=1)

And here are the row sums:

In [ ]:
mapslices(sum,A,dims=2)

### 7.5: Writing Fast Code

The rest of this chapter talks about important aspects of writing fast code.  We will walk through how to sum the first $n$ whole numbers in the fastest way.  In each case, we will make a function (called `sum1, sum2, ...`) and then test the overall speed of the function and discuss why features are faster than others. We will use the macro `@time` which will determine the time it takes to run. 

#### Sum Function 1
Consider first a for loop:

In [ ]:
function sum1(n::Int)
  local arr = collect(1:n)
  local sum = 0
  for i in arr
    sum += i
  end
  sum
end

This function takes in a positive number `n`, creates an array and then sums the elements of the array as a for loop.  We do some tests with this using the `@time` macro:

In [ ]:
@time sum1(1_000_000)

In [ ]:
@time sum1(10_000_000)

In [ ]:
@time sum1(100_000_000)

In [ ]:
@time sum1(1_000_000_000)

Notice though that in parentheses, it says the number of allocations.  Since the first one had an array of 1 million 64-bit integers (or 8 bytes), it is almost 8 megabytes. 

The factor of 100 higher created an array of 762 megabytes, which is not insignificant.  In short, it is expensive to allocation memory.

In [ ]:
function sum2(n::Integer)
    local sum = 0
    for i=1:n
        sum+=i
    end
    sum
end

This function doesn't use an array, since we don't really need one.  Let's see what happens

In [ ]:
@time sum2(100_000_000)

In [ ]:
@time sum2(1_000_000_000)

In [ ]:
@time sum2(10_000_000_000)

Why is this one faster than `sum1` ? What's going on with the last one?

In [ ]:
function sum3(n::Int)
    local sum = big(0)
    for i=1:n
        sum+=i
    end
    sum
end

If you said "overflow" in the above section, you win a prize--although I don't have a prize to give. :( 

Generally, if overflow is a problem, let's switch to `BigInt`s like above

In [ ]:
@time sum3(1_000_000)

In [ ]:
@time sum3(100_000_000)

We aren't going to have overflow problems, but you should notice that it is much slower to do operations with `BigInt`s. 

#### Exercise
Write a function similar to `sum3` however use `Int128` as the result (this should be the zero for local sum variable.) Call this `sum4` and time it comparsed to both `sum2` and `sum3`. 

Let's try using the `reduce` function:

In [ ]:
function sum5(n::Int) 
  reduce(+,1:big(n))
end

In [ ]:
@time sum5(1_000_000)

In [ ]:
@time sum5(10_000_000)

Note, this is much slower than the `sum3` method.  Let's try the built-in `sum` function:

In [ ]:
@time sum(1:big(10)^6)

In [ ]:
@time sum(1:big(10)^20)

In [ ]:
@time sum(1:big(10)^40)

In [ ]:
@time sum(1:big(10)^100)

What's going on?

### Summary of fast code:

* Stick with `Int64` if possible.  Always faster than `BigInt`
* don't create an array unless you need to.  Allocating memory is a slow process.
* use the built-in methods whenever possible.  They have been optimized. Julia is often super smart about some operations.

### 7.7: Computing Fibonacci Numbers

Let's look at the fibonacci numbers.  If $f_1=1,f_2=1$, then 
$$f_n=f_{n-1}+f_{n-2}\qquad\text{for $n\geq2$}$$

Let's find a fibonacci function that is recursive:

In [ ]:
function fibonacci(n::Integer)
    if n <= 2
        return 1
    else
        return fibonacci(n-1) + fibonacci(n-2)
    end
end

The first 10 can be found in the following way:

In [ ]:
map(fibonacci,1:10)

An alternative is:

In [ ]:
fibonacci(n::Int) = n<=2 ? 1 : fibonacci(n-1) + fibonacci(n-2)

This seems reasonable, but if we find the 40th one:

In [ ]:
@time fibonacci(40)

In [ ]:
@time fibonacci(41)

In [ ]:
@time fibonacci(42)

This isn't looking good. I'm sure (without trying we can find the 100th one.) Why is this so slow?

We going to see how many function evaluations are made.  Consider the adapted fibonacci code to compute the number of times it is evaluated:

In [ ]:
function fibonacciEval(n::Integer) 
  global num_evals
  if n==1 || n==2
    num_evals +=1
    return 1 
  else
    num_evals += 2
    return fibonacciEval(n-1) + fibonacciEval(n-2)
  end
end

In [ ]:
num_evals=0
fibonacciEval(5)
num_evals

In [ ]:
num_evals=0
fibonacciEval(20)
num_evals

In [ ]:
num_evals=0
fibonacciEval(21)
num_evals

Consider the following fibonacci based on a for loop:

In [ ]:
function fibonacci2(n)
  local x,y = (1,1)
  for i = 1:n-1
    x,y = (y, x+y)
  end
  x
end

In [ ]:
@time fibonacci2(50)

In [ ]:
@time fibonacci2(100)

#### Summary of Recursive functions
- Often writing recursive functions is easy, especially if that is the way they are defined. 
- However a short recursive function is not necessarily fast. 